# Preparing and preprocessing data

## 1. Import

In [ ]:
import os

import scanpy as sc
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
output_directory = 'cNMF_w_filtered_genes'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

## 2. Load atlas

In [ ]:
adata = sc.read_h5ad("../../../../datasets/Marteau_2024_CRC/core_atlas-adata.h5ad")

## 3. Pre-processing

### 3.1 Filtering cohorts

#### 3.1.1 Identify cohorts

In [ ]:
cohorts = adata.obs['dataset'].unique()

In [ ]:
print(f"Number of unique cohorts: {len(cohorts)}")

#### 3.1.2 Filter out cohorts with no myeloids

In [ ]:
valid_cohorts = []
for cohort in cohorts:
    subset = adata[adata.obs['dataset'] == cohort]
    if subset.obs['cell_type_coarse'].isin(['Myeloid cell', 'Neutrophil', 'Mast cell']).any():
        valid_cohorts.append(cohort)

In [ ]:
adata = adata[adata.obs['dataset'].isin(valid_cohorts)]

In [ ]:
print(f"Number of cohorts with myeloids: {len(valid_cohorts)}")

### 3.2 Filtering samples

#### 3.2.1 Keeping only tumor samples

In [ ]:
adata.obs['sample_type'].unique().tolist()

In [ ]:
print(f"Original shape: {adata.shape}")
adata = adata[adata.obs['sample_type'] == 'tumor']
print(f"New shape: {adata.shape}")

#### 3.2.2 Removing treated patients

In [ ]:
adata.obs['treatment_status_before_resection'].unique().tolist()

In [ ]:
print(f"Original shape: {adata.shape}")
adata = adata[adata.obs['treatment_status_before_resection'] != 'treated']
print(f"New shape: {adata.shape}")

### 3.3 Keep only myeloids

In [ ]:
adata = adata[adata.obs['cell_type_coarse'].isin(['Myeloid cell', 'Neutrophil', 'Mast cell'])]

#### 3.3.1 Filter out Platalet and Myeloid progenitor

Myeloid progenitors -> Mostly in metastasis (and some tumor) samples from treated patients

Platalets -> only found in blood samples

In [ ]:
adata = adata[~adata.obs['cell_type_fine'].isin(['Platelet', 'Myeloid progenitor'])]

In [ ]:
adata.obs['cell_type_fine'].unique().tolist()

#### 3.3.2 Verify 20 cells for each myeloid-type

In [ ]:
final_cohorts = []
for cohort in adata.obs['dataset'].unique():
    subset = adata[adata.obs['dataset'] == cohort]
    counts = subset.obs['cell_type_fine'].value_counts()
    if (counts >= 20).all():
        final_cohorts.append(cohort)

In [ ]:
print(f"Number of cohorts with at least 20 cells for each myeloid-type: {len(final_cohorts)}")

In [ ]:
final_cohorts

In [ ]:
# Get the list of unique datasets
# cohorts = adata.obs['dataset'].unique()

print("Checking for cell types with fewer than 20 cells per dataset:\n")

for cohort in final_cohorts:
    # Filter cells belonging to the current dataset
    adata_subset = adata.obs[adata.obs['dataset'] == cohort]

    # Count number of cells per cell type
    counts = adata_subset['cell_type_fine'].value_counts().sort_index()

    # Find underrepresented cell types (< 20 cells)
    underrepresented = counts[counts < 20]

    if not underrepresented.empty:
        print(f"Dataset '{cohort}' has cell types with fewer than 20 cells:")
        for cell_type, count in underrepresented.items():
            print(f"   - {cell_type:30} {count:>5} cells")

In [ ]:
adata = adata[adata.obs['dataset'].isin(final_cohorts)]

In [ ]:
adata.obs['cell_type_fine'].unique().tolist() 
# to show that no cohort has Eosinophil or Granulocyte progenitor 

### 3.4 Filtering cells and genes

In [ ]:
print(f"adata shape: {adata.shape}")

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)

#### 3.4.1 Check which genes are left now among the 2000 hvg

In [ ]:
adata_200 = adata.copy()

In [ ]:
sc.pp.filter_genes(adata_200, min_cells=200)

In [ ]:
print(f"adata shape: {adata_200.shape}")

In [ ]:
sc.pp.highly_variable_genes(adata_200, n_top_genes=2000, flavor='seurat')

In [ ]:
hvg_ensg_200 = adata_200.var_names[adata_200.var['highly_variable']]
hvg_symbols_200 = adata_200.var.loc[hvg_ensg_200, 'GeneSymbol'].tolist()
hvg_symbols_sorted_200 = sorted(hvg_symbols_200)

In [ ]:
genes_to_check = [
    "CD3D", "TRBV27", "TRAV21", "TRAC", "TRAV22", # T cells
    "IGKC", "IGHA1", "JCHAIN", "IGKV1-5", "IGLC1", # Plasma / B cells
    "SPRR1B", "KRT6A", "KRTDAP", "KRT14", "IVL", "KRT13", "SPRR2D" # Epithelial cells
]

present_genes = [gene for gene in genes_to_check if gene in hvg_symbols_sorted_200]
missing_genes = [gene for gene in genes_to_check if gene not in hvg_symbols_sorted_200]

print("Present:")
print(present_genes)
print("\nMissing:")
print(missing_genes)

With this first filtering we got rid of epithelial confounding genes, but still we have plasma and T cells genes.

In [ ]:
# save
filtered_counts_dir = os.path.join(output_directory, 'filtered_counts.h5ad')
sc.write(filtered_counts_dir, adata_200)

#### 3.4.2 Increase min_cells parameter

In [ ]:
adata_1000 = adata.copy()

In [ ]:
sc.pp.filter_genes(adata_1000, min_cells=1000)

In [ ]:
print(f"adata shape: {adata_1000.shape}")

In [ ]:
sc.pp.highly_variable_genes(adata_1000, n_top_genes=2000, flavor='seurat')

In [ ]:
hvg_ensg_1000 = adata_1000.var_names[adata_1000.var['highly_variable']]
hvg_symbols_1000 = adata_1000.var.loc[hvg_ensg_1000, 'GeneSymbol'].tolist()
hvg_symbols_sorted_1000 = sorted(hvg_symbols_1000)

In [ ]:
genes_to_check = [
    "CD3D", "TRBV27", "TRAV21", "TRAC", "TRAV22", # T cells
    "IGKC", "IGHA1", "JCHAIN", "IGKV1-5", "IGLC1", # Plasma / B cells
    "SPRR1B", "KRT6A", "KRTDAP", "KRT14", "IVL", "KRT13", "SPRR2D" # Epithelial cells
]

present_genes = [gene for gene in genes_to_check if gene in hvg_symbols_sorted_1000]
missing_genes = [gene for gene in genes_to_check if gene not in hvg_symbols_sorted_1000]

print("Present:")
print(present_genes)
print("\nMissing:")
print(missing_genes)

Increasing significantly the parameter does not change outcome.

## 4. Checking which cells are expressing non-myeloid-specific genes

In [ ]:
adata_200.obs['cell_type_fine'].value_counts()

In [ ]:
symbol_to_ensg = {
    symbol: ensg for ensg, symbol in zip(adata_200.var_names, adata_200.var["GeneSymbol"])
    if symbol in present_genes
}

ensg_genes = list(symbol_to_ensg.values())

is_expressed = (adata_200[:, ensg_genes].X > 0).toarray()

### 4.1 Grouping per cell type

In [ ]:
df_celltype = pd.DataFrame(is_expressed, columns=present_genes, index=adata_200.obs_names)
df_celltype["cell_type_fine"] = adata_200.obs["cell_type_fine"].values

# for each gene, count how many cells are expressing it (grouped per cell type)
expression_summary_celltype = df_celltype.groupby("cell_type_fine")[present_genes].sum().astype(int)

expression_summary_celltype = expression_summary_celltype.loc[:, expression_summary_celltype.sum().sort_values(ascending=False).index]
expression_summary_celltype = expression_summary_celltype.loc[expression_summary_celltype.sum(axis=1).sort_values(ascending=False).index]

In [ ]:
total_cells_per_type = adata_200.obs['cell_type_fine'].value_counts()
percent_expression_celltype = expression_summary_celltype.div(total_cells_per_type, axis=0) * 100

percent_expression_celltype = percent_expression_celltype.loc[:, percent_expression_celltype.sum().sort_values(ascending=False).index]
percent_expression_celltype = percent_expression_celltype.loc[percent_expression_celltype.sum(axis=1).sort_values(ascending=False).index]

In [ ]:
plt.figure(figsize=(12, 8))

sns.heatmap(percent_expression_celltype, cmap="viridis", annot=True, fmt=".1f", cbar_kws={'label': '% cells expressing'})

plt.title("Heatmap: % of cells expressing each gene per cell type")
plt.xlabel("Genes")
plt.ylabel("Cell type (cell_type_fine)")
plt.tight_layout()
plt.show()

### 4.2 Grouping per cohort

In [ ]:
df_cohorts = pd.DataFrame(is_expressed, columns=present_genes, index=adata_200.obs_names)
df_cohorts["dataset"] = adata_200.obs["dataset"].values

# for each gene, count how many cells are expressing it (grouped per cohorts)
expression_summary_cohorts = df_cohorts.groupby("dataset")[present_genes].sum().astype(int)

expression_summary_cohorts = expression_summary_cohorts.loc[:, expression_summary_cohorts.sum().sort_values(ascending=False).index]
expression_summary_cohorts = expression_summary_cohorts.loc[expression_summary_cohorts.sum(axis=1).sort_values(ascending=False).index]

In [ ]:
total_cells_per_cohort = adata_200.obs['dataset'].value_counts()
percent_expression_cohorts = expression_summary_cohorts.div(total_cells_per_cohort, axis=0) * 100

percent_expression_cohorts = percent_expression_cohorts.loc[:, percent_expression_cohorts.sum().sort_values(ascending=False).index]
percent_expression_cohorts = percent_expression_cohorts.loc[percent_expression_cohorts.sum(axis=1).sort_values(ascending=False).index]

In [ ]:
plt.figure(figsize=(12, 8))

sns.heatmap(percent_expression_cohorts, cmap="viridis", annot=True, fmt=".1f", cbar_kws={'label': '% cells expressing'})

plt.title("Heatmap: % of cells expressing each gene per cohort")
plt.xlabel("Genes")
plt.ylabel("Cohorts")
plt.tight_layout()
plt.show()

### 4.3 Grouping per cell type and cohort

In [ ]:
df_both = pd.DataFrame(is_expressed, columns=present_genes, index=adata_200.obs_names)
df_both["cell_type_fine"] = adata_200.obs["cell_type_fine"].values
df_both["dataset"] = adata_200.obs["dataset"].values

expression_summary_both = df_both.groupby(["cell_type_fine", "dataset"])[present_genes].sum().astype(int)

expression_summary_both = expression_summary_both.loc[
    expression_summary_both.sum(axis=1).sort_values(ascending=False).index,
    expression_summary_both.sum().sort_values(ascending=False).index
]

expression_summary_both = expression_summary_both.sort_index(level=["cell_type_fine", "dataset"])

In [ ]:
total_cells_per_group = df_both.groupby(["cell_type_fine", "dataset"]).size()

percent_expression_both = expression_summary_both.div(total_cells_per_group, axis=0) * 100

percent_expression_both = percent_expression_both.loc[
    percent_expression_both.sum(axis=1).sort_values(ascending=False).index,
    percent_expression_both.sum().sort_values(ascending=False).index
]

percent_expression_both = percent_expression_both.sort_index(level=["cell_type_fine", "dataset"])

In [ ]:
for cell_type in percent_expression_both.index.get_level_values(0).unique():

    # Just that cell type
    df_plot = percent_expression_both.loc[cell_type]

    # rename
    df_plot.index = [f"{cohort}" for cohort in df_plot.index]

    plt.figure(figsize=(12, max(2, 0.5 * len(df_plot))))  # adatta l'altezza al numero di coorti
    sns.heatmap(df_plot, cmap="viridis", annot=True, fmt=".1f", cbar_kws={'label': '% cells expressing'})

    plt.title(f"Heatmap: % cells expressing per gene — {cell_type}")
    plt.xlabel("Genes")
    plt.ylabel("Cohort")
    plt.tight_layout()
    plt.show()
